# Creating the Gene Table
This notebook is copied from the [Pymodulon GitHub repository](https://github.com/SBRG/pymodulon/blob/master/docs/tutorials/creating_the_gene_table.ipynb)

## Get information from GFF files

In [1]:
from pymodulon.gene_util import *
import os

First, download the FASTA and GFF files for your organism and its plasmids from NCBI.

Enter the location of all your GFF files here:

In [2]:
gff_files = [os.path.join('saureus.gff')]

The following cell will convert all the GFF files into a single Pandas DataFrame for easy manipulation. Pseudogenes have multiple rows in a GFF file (one for each fragment), but only the first fragment will be kept.

In [3]:
keep_cols = ['accession','start','end','strand','gene_name','old_locus_tag','gene_product','ncbi_protein']

DF_annot = gff2pandas(gff_files,index='locus_tag')
DF_annot = DF_annot[keep_cols]

DF_annot.head()

,accession,start,end,strand,gene_name,old_locus_tag,gene_product,ncbi_protein
locus_tag,,,,,,,,
SAOUHSC_00001,NC_007795.1,517.0,1878.0,+,dnaA,None,chromosomal replication initiation protein,YP_498609.1
SAOUHSC_00002,NC_007795.1,2156.0,3289.0,+,None,None,DNA polymerase III subunit beta,YP_498610.1
SAOUHSC_00003,NC_007795.1,3670.0,3915.0,+,None,None,hypothetical protein,YP_498611.1
SAOUHSC_00004,NC_007795.1,3912.0,5024.0,+,recF,None,recombination protein F,YP_498612.1
SAOUHSC_00005,NC_007795.1,5034.0,6968.0,+,None,None,DNA gyrase subunit B,YP_498613.1


To ensure that the gene index used is identical to the expression matrix, load in your data.

In [4]:
log_tpm_file = os.path.join('log_tpm.csv')
DF_log_tpm = pd.read_csv(log_tpm_file,index_col=0)
DF_log_tpm.head()

,DRX300641,DRX300642,DRX300643,ERX1222798,ERX1222799,ERX1222800,ERX2826862,ERX2826863,ERX2826864,ERX2826865,...,SRX9634010,SRX9634011,SRX9634012,SRX9634015,SRX9634016,SRX9634017,SRX9634018,SRX9634019,SRX9634020,SRX965931
Geneid,,,,,,,,,,,,,,,,,,,,,
SAOUHSC_00001,8.075960,7.913237,8.103539,8.231650,7.480144,7.833791,8.829448,8.936380,8.989139,8.942718,...,8.813741,8.461149,9.068514,8.446301,8.447756,8.781591,8.768944,8.520810,8.565999,8.558046
SAOUHSC_00002,8.491571,8.599078,8.401215,9.433116,8.726581,9.329891,8.962073,9.160966,9.290196,8.992331,...,9.093600,8.482311,8.937456,8.529349,8.782375,9.033129,9.018110,8.161739,8.052105,8.127758
SAOUHSC_00003,9.304385,8.849633,8.676897,7.504135,8.411847,8.858987,8.747960,8.976711,8.626191,8.742656,...,8.714124,8.020151,8.725003,8.142546,7.577228,7.810381,7.797447,7.668054,7.790997,7.671018
SAOUHSC_00004,9.119361,9.325414,8.971591,9.298017,9.538484,9.230754,8.412644,8.391423,8.605931,8.362605,...,9.841963,9.298584,9.323726,9.137359,8.794212,9.041146,9.083412,9.373125,9.467331,9.312853
SAOUHSC_00005,9.548104,9.573234,9.433046,9.791319,10.199322,9.799053,8.949354,8.885628,9.152409,8.849877,...,8.902671,8.890497,9.024617,8.658290,8.178211,8.635210,8.599545,9.773655,9.854817,9.969962


Check that the genes are the same in the expression dataset as in the annotation dataframe. Mismatched genes are listed below.

In [5]:
test = DF_annot.sort_index().index == DF_log_tpm.sort_index().index
DF_annot[~test]

,accession,start,end,strand,gene_name,old_locus_tag,gene_product,ncbi_protein
locus_tag,,,,,,,,


## (Optional) KEGG and COGs

### Generate nucleotide fasta files for CDS

Enter the location of all your fasta files here:

In [6]:
fasta_files = [os.path.join('saureus.fna')]

The following code generates CDS files using your FASTA and GFF3 files

In [7]:
from Bio import SeqIO

cds_list = []
for fasta in fasta_files:
    seq = SeqIO.read(fasta,'fasta')

    # Get gene information for genes in this fasta file
    df_genes = DF_annot[DF_annot.accession == seq.id]
    
    for i,row in df_genes.iterrows():
        cds = seq[int(row.start)-1:int(row.end)] #Added int() heredue to errors
        if row.strand == '-':
            cds = seq[int(row.start)-1:int(row.end)].reverse_complement()  #Added int() heredue to errors
        cds.id = row.name
        cds.description = row.gene_name if pd.notnull(row.gene_name) else row.name
        cds_list.append(cds)

In [8]:
cds_list[:5]

[SeqRecord(seq=Seq('ATGTCGGAAAAAGAAATTTGGGAAAAAGTGCTTGAAATTGCTCAAGAAAAATTA...TAA'), id='SAOUHSC_00001', name='NC_007795.1', description='dnaA', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGATGGAATTCACTATTAAAAGAGATTATTTTATTACACAATTAAATGACACA...TAA'), id='SAOUHSC_00002', name='NC_007795.1', description='SAOUHSC_00002', dbxrefs=[]),
 SeqRecord(seq=Seq('GTGATTATTTTGGTTCAAGAAGTTGTAGTAGAAGGAGACATTAATTTAGGTCAA...TGA'), id='SAOUHSC_00003', name='NC_007795.1', description='SAOUHSC_00003', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGAAGTTAAATACACTCCAATTAGAAAATTATCGTAACTATGATGAGGTTACG...TAA'), id='SAOUHSC_00004', name='NC_007795.1', description='recF', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGGTGACTGCATTGTCAGATGTAAACAACACGGATAATTATGGTGCTGGGCAA...TAA'), id='SAOUHSC_00005', name='NC_007795.1', description='SAOUHSC_00005', dbxrefs=[])]

Save the CDS file

In [9]:
cds_file = os.path.join('CDS_files','CDS.fna')
SeqIO.write(cds_list, cds_file, 'fasta')

2767

### Run EggNOG Mapper
1. Go to http://eggnog-mapper.embl.de/.
1. Upload the CDS.fna file from your organism directory (within the sequence_files folder)
1. Make sure to limit the taxonomy to the correct level
1. After the job is submitted, you must follow the link in your email to run the job.
1. Once the job completes (after ~4 hrs), download the annotations file.
1. Save the annotation file

### Get KEGG IDs

Once you have the EggNOG annotations, load the annotation file

In [10]:
eggnog_file = os.path.join('eggNOG','MM_pqq12tbj.emapper.annotations.txt')

In [11]:
DF_eggnog = pd.read_csv(eggnog_file,sep='\t',skiprows=4,header=None)
eggnog_cols = ['query_name','seed eggNOG ortholog','seed ortholog evalue','seed ortholog score',
               'Predicted taxonomic group','Predicted protein name','Gene Ontology terms',
               'EC number','KEGG_orth','KEGG_pathway','KEGG_module','KEGG_reaction',
               'KEGG_rclass','BRITE','KEGG_TC','CAZy','BiGG Reaction','tax_scope',
               'eggNOG OGs','bestOG_deprecated','COG']  #deleted 'eggNOG free text description' column at the end due to bug

DF_eggnog.columns = eggnog_cols

# Strip last three rows as they are comments
DF_eggnog = DF_eggnog.iloc[:-3]

# Set locus tag as index
DF_eggnog = DF_eggnog.set_index('query_name')
DF_eggnog.index.name = 'locus_tag'

DF_eggnog.head()

,seed eggNOG ortholog,seed ortholog evalue,seed ortholog score,Predicted taxonomic group,Predicted protein name,Gene Ontology terms,EC number,KEGG_orth,KEGG_pathway,KEGG_module,KEGG_reaction,KEGG_rclass,BRITE,KEGG_TC,CAZy,BiGG Reaction,tax_scope,eggNOG OGs,bestOG_deprecated,COG
locus_tag,,,,,,,,,,,,,,,,,,,,
#query,seed_ortholog,evalue,score,eggNOG_OGs,max_annot_lvl,COG_category,Description,Preferred_name,GOs,EC,KEGG_ko,KEGG_Pathway,KEGG_Module,KEGG_Reaction,KEGG_rclass,BRITE,KEGG_TC,CAZy,BiGG_Reaction,PFAMs
SAOUHSC_00001,525378.HMPREF0793_1044,1.94e-280,772.0,"COG0593@1|root,COG0593@2|Bacteria,1TPV7@1239|F...",91061|Bacilli,L,it binds specifically double-stranded DNA at a...,dnaA,"GO:0003674,GO:0003676,GO:0003677,GO:0003688,GO...",-,ko:K02313,"ko02020,ko04112,map02020,map04112",-,-,-,"ko00000,ko00001,ko03032,ko03036",-,-,-,"Bac_DnaA,Bac_DnaA_C,DnaA_N"
SAOUHSC_00002,1280.SAXN108_0003,4.23e-268,734.0,"COG0592@1|root,COG0592@2|Bacteria,1TQ7J@1239|F...",91061|Bacilli,L,Confers DNA tethering and processivity to DNA ...,dnaN,-,2.7.7.7,ko:K02338,"ko00230,ko00240,ko01100,ko03030,ko03430,ko0344...",M00260,"R00375,R00376,R00377,R00378",RC02795,"ko00000,ko00001,ko00002,ko01000,ko03032,ko03400",-,-,-,"DNA_pol3_beta,DNA_pol3_beta_2,DNA_pol3_beta_3"
SAOUHSC_00003,1280.SAXN108_0004,1.56e-49,157.0,"COG2501@1|root,COG2501@2|Bacteria,1VEJ2@1239|F...",91061|Bacilli,S,S4 domain,yaaA,-,-,ko:K14761,-,-,-,-,"ko00000,ko03009",-,-,-,S4_2
SAOUHSC_00004,1280.SAXN108_0005,6.45e-264,723.0,"COG1195@1|root,COG1195@2|Bacteria,1TP9U@1239|F...",91061|Bacilli,L,it is required for DNA replication and normal ...,recF,"GO:0000731,GO:0005575,GO:0005622,GO:0005623,GO...",-,ko:K03629,"ko03440,map03440",-,-,-,"ko00000,ko00001,ko03400",-,-,-,SMC_N


Now we will pull the KEGG information from the eggNOG file, including orthology, pathway, module, and reactions for each gene.

In [12]:
DF_kegg = DF_eggnog[['KEGG_orth','KEGG_pathway','KEGG_module','KEGG_reaction']]

# Melt dataframe
DF_kegg = DF_kegg.reset_index().melt(id_vars='locus_tag') 

# Remove null values
DF_kegg = DF_kegg[DF_kegg.value.notnull()]

# Split comma-separated values into their own rows
list2struct = []
for name,row in DF_kegg.iterrows():
    for val in row.value.split(','):
        list2struct.append([row.locus_tag,row.variable,val])

DF_kegg = pd.DataFrame(list2struct,columns=['gene_id','database','kegg_id'])

# Remove ko entries, as only map entries are searchable in KEGG pathway
DF_kegg = DF_kegg[~DF_kegg.kegg_id.str.startswith('ko')]

DF_kegg.head()

,gene_id,database,kegg_id
0,#query,KEGG_orth,Preferred_name
1,SAOUHSC_00001,KEGG_orth,dnaA
2,SAOUHSC_00002,KEGG_orth,dnaN
3,SAOUHSC_00003,KEGG_orth,yaaA
4,SAOUHSC_00004,KEGG_orth,recF


### Save KEGG information

In [13]:
DF_kegg.to_csv(os.path.join('KEGG','kegg_mapping.csv'))

### Save COGs to annotation dataframe

In [14]:
DF_annot['COG'] = DF_eggnog.COG

# Make sure COG only has one entry per gene
DF_annot['COG'] = [item[0] if isinstance(item,str) else item for item in DF_annot['COG']]

## Uniprot ID mapping

The ``uniprot_id_mapping`` function is a python wrapper for the [Uniprot ID mapping tool](https://www.uniprot.org/uploadlists/). Use ``input_id=P_REFSEQ_AC`` if the FASTA/GFF files are from RefSeq, and ``input_id=EMBL`` if the files are from Genbank.

In [36]:
#mapping_uniprot = uniprot_id_mapping(DF_annot.ncbi_protein.fillna(''),input_id='EMBL',output_id='ACC',
#                             input_name='ncbi_protein',output_name='uniprot')
#mapping_uniprot.head()


#mapping_uniprot = uniprot_id_mapping(DF_annot.ncbi_protein.fillna(''),input_id='P_REFSEQ_AC',output_id='ACC',
#                                     input_name='ncbi_protein',output_name='uniprot')
#mapping_uniprot.head()

import json
import requests
import time

URL = 'https://rest.uniprot.org/idmapping'
IDS = DF_annot.ncbi_protein.fillna('').values.tolist()


params = {
   'from': 'UniProtKB_AC-ID',
   'to': 'ChEMBL',
   'ids': ' '.join(IDS)
}

response = requests.post(f'{URL}/run', params)
job_id = response.json()['jobId']
job_status = requests.get(f'{URL}/status/{job_id}')
d = job_status.json()

# Make three attemps to get the results
for i in range(3):
    if d.get("job_status") == 'FINISHED' or d.get('results'):
        job_results = requests.get(f'{URL}/results/{job_id}')
        results = job_results.json()
        for obj in results['results']:
            print(f'{obj["from"]}\t{obj["to"]}')
        break
    time.sleep(1)
#print(IDS)
#print(response.json())
print(job_status.json())

{'results': [], 'failedIds': ['YP_498609.1 YP_498610.1 YP_498611.1 YP_498612.1 YP_498613.1 YP_498614.1 YP_498615.1 YP_498616.1 YP_498617.1 YP_498618.1 YP_498619.1 YP_498620.1 YP_498621.1 YP_498622.1 YP_498623.1 YP_498624.1 YP_498625.1 YP_498626.1 YP_498627.1 YP_498628.1 YP_498629.1 YP_498630.1 YP_498631.1 YP_498632.1 YP_498633.1 YP_498634.1 YP_498635.1 YP_498636.1 YP_498637.1 YP_498638.1 YP_498639.1 YP_498640.1 YP_498641.1 YP_498642.1 YP_498643.1 YP_498644.1 YP_498645.1 YP_498646.1 YP_498647.1 YP_498648.1 YP_498649.1 YP_498650.1 YP_498651.1 YP_498652.1 YP_498653.1 YP_498654.1 YP_498655.1 YP_498656.1 YP_498657.1 YP_498658.1 YP_498659.1 YP_498660.1 YP_498661.1 YP_498662.1 YP_498663.1 YP_498664.1 YP_498665.1 YP_498666.1 YP_498667.1 YP_498668.1 YP_498669.1 YP_498670.1 YP_498671.1 YP_498672.1 YP_498673.1 YP_498674.1 YP_498675.1 YP_498676.1 YP_498677.1 YP_498678.1 YP_498679.1 YP_498680.1 YP_498681.1 YP_498682.1 YP_498683.1 YP_498685.1 YP_498686.1 YP_498687.1 YP_498688.1 YP_498689.1 YP_498690

In [23]:
# Merge with current annotation
DF_annot = pd.merge(DF_annot.reset_index(),mapping_uniprot,how='left',on='ncbi_protein')
DF_annot.set_index('locus_tag',inplace=True)
DF_annot.head()

NameError: name 'mapping_uniprot' is not defined

## Add Biocyc Operon information

To obtain operon information from Biocyc, follow the steps below

1. Go to [Biocyc.org](https://biocyc.org/) (you may need to create an account and/or login)
2. Change the organism database to your organism/strain
3. Select **SmartTables** -> **Special SmartTables**
4. Select **"All genes of \<organism\>"**
5. Select the **"Gene Name"** column
6. Under **"ADD TRANSFORM COLUMN"** select **"Genes in same transcription unit"**
7. Select the **"Genes in same transcription unit"** column
8. Under **"ADD PROPERTY COLUMN"** select **"Accession-1"**
9. Under **OPERATIONS**, select **"Export"** -> **"to Spreadsheet File..."**
10. Select **"common names"** and click **"Export smarttable"**
11. Add file location below and run the code cell

In [ ]:
biocyc_file = os.path.join('..','data','external','biocyc_annotations.txt')

DF_biocyc = pd.read_csv(biocyc_file,sep='\t')

# Remove genes with no accession
DF_biocyc = DF_biocyc[DF_biocyc['Accession-1'].notnull()]

# Set the accession (i.e. locus tag) as index
DF_biocyc = DF_biocyc.set_index('Accession-1').sort_values('Left-End-Position')

# Specific for B. subtilis: Fix locus tags
DF_biocyc.index = DF_biocyc.index.str.replace('BSU','BSU_')

# Only keep genes in the final annotation file
DF_biocyc = DF_biocyc.reindex(DF_annot.index)

# Reformat transcription units
DF_biocyc['operon_list'] = DF_biocyc['Accession-1.1'].apply(reformat_biocyc_tu)

# Fill None with locus tags
DF_biocyc['operon_list'].fillna(DF_biocyc.index.to_series(), inplace=True)

DF_biocyc.head()

### Assign unique IDs to operons

The following code assigns unique names to each operon

In [ ]:
# Get all operons
operons = DF_biocyc['operon_list'].unique()

# Map each operon to a unique string
operon_dict = {operon: "Op"+str(i) for i, operon in enumerate(operons)}

# Add names to dataframe
DF_biocyc['operon'] = [operon_dict[op] for op in DF_biocyc["operon_list"]]

DF_biocyc.head()

Finally, merge the Biocyc information with the main annotation DataFrame

In [ ]:
DF_annot['operon'] = DF_biocyc['operon']

## Clean up and save annotation

First, we will re-order the annotation columns

In [ ]:
if 'old_locus_tag' in DF_annot.columns:
    order = ['gene_name','accession','old_locus_tag','start','end','strand','gene_product','COG','uniprot','operon']
else:
    order = ['gene_name','accession','start','end','strand','gene_product','COG','uniprot','operon']
    
DF_annot = DF_annot[order]

In [ ]:
DF_annot.head()

## Final statistics

The following graphs show how much information is available for the organism.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style('ticks')

In [ ]:
fig,ax = plt.subplots()
DF_annot.count().plot(kind='bar',ax=ax)
ax.set_ylabel('# of Values',fontsize=18)
ax.tick_params(labelsize=16)

## Fill missing values

Some organisms are missing gene names, so these will be filled with locus tag gene names.

In [ ]:
# Fill in missing gene names with locus tag names
DF_annot['tmp_name'] = DF_annot.copy().index.tolist()
DF_annot.gene_name.fillna(DF_annot.tmp_name,inplace=True)
DF_annot.drop('tmp_name',axis=1,inplace=True)

 COG letters will also be converted to the full name.

In [ ]:
# Fill missing COGs with X
DF_annot['COG'].fillna('X',inplace=True)

# Change single letter COG annotation to full description
DF_annot['COG'] = DF_annot.COG.apply(cog2str)

counts = DF_annot.COG.value_counts()
plt.pie(counts.values,labels=counts.index);

Uncomment the following line to save the gene annotation dataset

In [ ]:
DF_annot.to_csv(os.path.join('..','data','processed_data','gene_info.csv'))

## GO Annotations

To start, download the GO Annotations for your organism from AmiGO 2

1. Go to [AmiGO 2](http://amigo.geneontology.org/amigo/search/annotation)
1. Filter for your organism
1. Click ``CustomDL``
1. Drag ``GO class (direct)`` to the end of your Selected Fields
1. Enter the location of your GO annotation file below and run the following code block

In [ ]:
go_file = os.path.join('..','data','external','GO_annotations.txt')

In [ ]:
DF_GO = pd.read_csv(go_file,sep='\t',header=None,usecols=[2,17])
DF_GO.columns = ['gene_name','gene_ontology']
DF_GO.head()

Convert the gene names to gene locus tags, and drop gene names that cannot be converted

In [ ]:
name2num = {v:k for k,v in DF_annot.gene_name.to_dict().items()}

In [ ]:
DF_GO['gene_id'] = [name2num[x] if x in name2num.keys() else None for x in DF_GO.gene_name]

In [ ]:
DF_GO.head()

Now we remove null entries

In [ ]:
DF_GO = DF_GO[DF_GO.gene_id.notnull()]

In [ ]:
DF_GO.head()

Uncomment the line below to save the annotations

In [ ]:
DF_GO[['gene_id','gene_name','gene_ontology']].to_csv(os.path.join('..','data','external','GO_annotations_curated.csv'))